In [ ]:
## RetinaFace

# load CUDA so PyTorch will build for the right version
module load conda/23.11.0      # or whatever is in `module avail` in cuda

# create and activate a conda env
conda create -n facedet python=3.10 -y
conda activate facedet

# install InsightFace + CPU ONNX Runtime + OpenCV
conda install -c conda-forge insightface onnxruntime opencv pandas tqdm -y

In [ ]:
mkdir -p frames/movie1
ffmpeg -i movie1.mp4 -vf fps=1 frames/movie1/%05d.jpg

In [ ]:
#verify the instalation

python - <<'PY'
from insightface.app import FaceAnalysis
import onnxruntime as ort, cv2, numpy as np, os, sys
print("InsightFace version:", sys.modules['insightface'].__version__)
print("ONNX Runtime providers:", ort.get_available_providers())
app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
app.prepare(det_size=(640,640))
print("Loaded detection model OK ✔")
PY


In [ ]:
#make project tree

mkdir -p ~/face_detect_project/{videos,frames,results,scripts,logs}
cd ~/face_detect_project

In [ ]:
# pull in movies 

# movies live in ~/Desktop/Post_20140821_version/ on my desktop (you will need to house the movies)
ln -s ~/Desktop/Post_20140821_version/*.mp4 videos/
ls videos                   # sanity‑check: should list your .mp4 files

In [ ]:
# the detection script 

cat > scripts/detect_faces.py <<'PY'

#!/usr/bin/env python3
import cv2, glob, os, argparse, pandas as pd, tqdm
from insightface.app import FaceAnalysis

p = argparse.ArgumentParser()
p.add_argument("--frames_dir", required=True)
p.add_argument("--out_csv",    required=True)
args = p.parse_args()

app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
app.prepare(ctx_id=-1, det_size=(640,640))

rows = []
for jpg in tqdm.tqdm(sorted(glob.glob(os.path.join(args.frames_dir, '*.jpg')))):
    sec = int(os.path.basename(jpg)[:-4])
    img = cv2.imread(jpg)
    h, w = img.shape[:2]
    faces = app.get(img)
    face_px = sum((f.bbox[2]-f.bbox[0])*(f.bbox[3]-f.bbox[1]) for f in faces)
    rows.append([sec, bool(faces), len(faces), face_px/(h*w)])

pd.DataFrame(rows, columns=['sec','face_present','num_faces','face_area']).to_csv(
    args.out_csv, index=False)
PY
chmod +x scripts/detect_faces.py


In [ ]:
#Slurm batch template 

cat > scripts/run_faces.sbatch <<'SB'
#!/bin/bash
#SBATCH --job-name=faces_${1}
#SBATCH --partition=Main          # CPU queue
#SBATCH --cpus-per-task=8
#SBATCH --time=01:30:00
#SBATCH --mem=10G
#SBATCH --output=logs/faces_${1}.%j.log

module load conda/23.11.0
conda activate facedet

python scripts/detect_faces.py \
       --frames_dir frames/${1} \
       --out_csv    results/${1}_faces.csv
SB

In [ ]:
# extract 1‑fps frames
module load ffmpeg/2203           # if ffmpeg isn’t already on the PATH
for f in videos/*.mp4; do
  name=$(basename "$f" .mp4)
  mkdir -p frames/"$name"
  ffmpeg -i "$f" -vf fps=1 frames/"$name"/%05d.jpg
done

# launch detection script
mkdir -p logs results
for f in videos/*.mp4; do
  name=$(basename "$f" .mp4)
  sbatch scripts/run_faces.sbatch "$name"
done


In [ ]:
#monitor progress

squeue -u $USER
tail -f logs/faces_*.log      # Ctrl‑C to stop tailing

#verify output when job is done 

head results/*_faces.csv | column -t -s,

In [ ]:
#finally, save movies to desktop 

# copy every per‑movie CSV
scp ms16@login-k01.hpc.youruni.edu:~/face_detect_project/results/*_faces.csv .